
# Quickstart guide

This example demonstrates how to build a simple content-based audio retrieval model and evaluate the retrieval accuracy on a small song dataset, CAL500. This dataset consists of 502 western pop songs, performed by 499 unique artists. Each song is tagged by at least three people using a standard survey and a fixed tag vocabulary of 174 musical concepts.

This package includes a loading utility for getting and processing this dataset, which makes loading quite easy.

In [1]:
from cbar.datasets import fetch_cal500

X, Y = fetch_cal500()

Calling `fetch_cal500()` initally downloads the CAL500 dataset to a subfolder of your home directory. You can specify a different location using the `data_home` parameter (`fetch_cal500(data_home='path')`). Subsequents calls simply load the dataset.

The raw dataset consists of about 10,000 39-dimensional features vectors
per minute of audio content which were created by

1. Sliding a half-overlapping short-time window of 12 milliseconds over each song's waveform data.
2. Extracting the 13 mel-frequency cepstral coefficients.
3. Appending the instantaneous first-order and second-order derivatives.

Each song is, then, represented by exactly 10,000 randomly subsampled, real-valued feature vectors as a *bag-of-frames*. The *bag-of-frames* features are further processed into one *k*-dimensional feature vector by encoding the feature vectors using a codebook and pooling them into one compact vector.

Specifically, *k*-means is used to cluster all frame vectors into *k* clusters. The resulting cluster centers correspond to the codewords in the codebook. Each frame vector is assigned to its closest cluster center and a song represented as the counts of frames assigned to each of the *k* cluster centers.

By default, `fetch_cal500()` uses a codebook size of 512 but this size is easily modified with the `codebook_size` parameter (`fetch_cal500(codebook_size=1024)`).

In [2]:
X.shape, Y.shape

((502, 512), (502, 174))

Let's split the data into training data and test data, fit the model on the training data, and evaluate it on the test data. Import and instantiate the model first.

In [3]:
from cbar.pamir import PAMIR

model = PAMIR()

Then split the data and fit the model using the training data.

In [4]:
from cbar.cross_validation import train_test_split_plus

(X_train, X_test,
 Y_train, Y_test,
 Q_vec, weights) = train_test_split_plus(X, Y)

%time model.fit(X_train, Y_train, Q_vec, X_test, Y_test)

/home/rocket/miniconda3/envs/cbar/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


iter:        0, P10: 0.193, AP: 0.189, loss: 0.000
iter:    10000, P10: 0.196, AP: 0.192, loss: 0.915
iter:    20000, P10: 0.182, AP: 0.192, loss: 0.629
iter:    30000, P10: 0.180, AP: 0.190, loss: 0.516
iter:    40000, P10: 0.191, AP: 0.193, loss: 0.434
iter:    50000, P10: 0.186, AP: 0.195, loss: 0.393
iter:    60000, P10: 0.179, AP: 0.193, loss: 0.349
iter:    70000, P10: 0.185, AP: 0.193, loss: 0.322
iter:    80000, P10: 0.183, AP: 0.194, loss: 0.295
iter:    90000, P10: 0.190, AP: 0.194, loss: 0.276
CPU times: user 2min 15s, sys: 1min 21s, total: 3min 37s
Wall time: 1min 36s


cbar.pamir.PAMIR(max_iter=100000, C=1.0, valid_interval=10000, max_dips=20)

Now, predict the scores for each query with all songs. Ordering the songs from highest to lowest score corresponds to the ranking.

In [5]:
Y_score = model.predict(Q_vec, X_test)

Evaluate the predictions.

In [6]:
from cbar.evaluation import Evaluator
from cbar.utils import make_relevance_matrix

n_relevant = make_relevance_matrix(Q_vec, Y_train).sum(axis=1)

evaluator = Evaluator()
evaluator.eval(Q_vec, weights, Y_score, Y_test, n_relevant)
evaluator.prec_at

defaultdict(list,
            {1: [0.19512195121951223],
             2: [0.18902439024390244],
             3: [0.19512195121951217],
             4: [0.17784552845528456],
             5: [0.17479674796747968],
             6: [0.1719512195121951],
             7: [0.1784843205574913],
             8: [0.18106126596980254],
             9: [0.17843592721641502],
             10: [0.17957801006581497],
             11: [0.1819165082884595],
             12: [0.18620635800513852],
             13: [0.18959128608518855],
             14: [0.1965277270155319],
             15: [0.21007529056309546],
             16: [0.213417908311201],
             17: [0.21588297970561243],
             18: [0.2227166952687039],
             19: [0.23173199265851988],
             20: [0.23776231236021314]})

## Cross-validation

The `cv` function in the `cross_validation` module offers an easy way to evaluate a retrieval method on multiple splits of the data. Let's run the same experiment on three folds.

In [7]:
from cbar.cross_validation import cv

In [ ]:
cv('cal500', 512, n_folds=3, method='pamir', valid_interval=1000)

2019-03-13 15:00:29,314 [MainThread  ] [INFO ]  Running CV with 3 folds ...
2019-03-13 15:00:52,956 [MainThread  ] [INFO ]  Validating fold 1 ...
iter:        0, P10: 0.162, AP: 0.172, loss: 0.000
iter:     1000, P10: 0.169, AP: 0.170, loss: 1.249
iter:     2000, P10: 0.168, AP: 0.172, loss: 1.106
iter:     3000, P10: 0.166, AP: 0.175, loss: 1.071
iter:     4000, P10: 0.173, AP: 0.176, loss: 0.909
iter:     5000, P10: 0.178, AP: 0.177, loss: 0.899
iter:     6000, P10: 0.175, AP: 0.178, loss: 0.885
iter:     7000, P10: 0.177, AP: 0.177, loss: 0.682
iter:     8000, P10: 0.175, AP: 0.179, loss: 0.673
iter:     9000, P10: 0.170, AP: 0.179, loss: 0.654
iter:    10000, P10: 0.170, AP: 0.179, loss: 0.716
iter:    11000, P10: 0.173, AP: 0.180, loss: 0.640
iter:    12000, P10: 0.174, AP: 0.180, loss: 0.556
iter:    13000, P10: 0.173, AP: 0.177, loss: 0.628
iter:    14000, P10: 0.172, AP: 0.177, loss: 0.554
iter:    15000, P10: 0.166, AP: 0.177, loss: 0.574
iter:    16000, P10: 0.168, AP: 0.176,

The cross-validation results including retrieval method parameters are written to a JSON file. For each dataset three separate result files for mean average precision (MAP), precision-at-*k*, and precision-at-10 as a function of relevant training examples are written to disk. Here are the mean average precision values of the last cross-validation run.

In [ ]:
import json
import os
from cbar.settings import RESULTS_DIR

results = json.load(open(os.path.join(RESULTS_DIR, 'cal500_mean_ap.json')))
results[list(results.keys())[-1]]['precision']

## Start cross-validation with the CLI

This package comes with a simple CLI which makes it easy to start cross-validation experiments from the command line. The CLI enables you to specify a dataset and a retrieval method as well as additional options in one line.

To start an experiment on the CAL500 dataset with the LORETA retrieval method, use the following command.

```
$ cbar crossval --dataset cal500 loreta 
```

This simple command uses all the default parameters for LORETA but you can specify all parameters as arguments to the `loreta` command. To see the available options for the `loreta` command, ask for help like this.

```
$ cbar crossval loreta --help
Usage: cbar crossval loreta [OPTIONS]

Options:
  -n, --max-iter INTEGER        Maximum number of iterations
  -i, --valid-interval INTEGER  Rank of parameter matrix W
  -k INTEGER                    Rank of parameter matrix W
  --n0 FLOAT                    Step size parameter 1
  --n1 FLOAT                    Step size parameter 2
  -t, --rank-thresh FLOAT       Threshold for early stopping
  -l, --lambda FLOAT            Regularization constant
  --loss [warp|auc]             Loss function
  -d, --max-dips INTEGER        Maximum number of dips
  -v, --verbose                 Verbosity
  --help                        Show this message and exit.
```